<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/taller5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=1WNLKH10YpQNNk9eeRIyYLwGkxNbNp-Mm" width="100%">

# **Taller 5**
---

En este taller se evaluarán los conocimientos adquiridos en análisis no supervisado de textos con modelos de agrupamiento y de tópicos. Para esto, usaremos un conjunto de datos de poemas en español.

Comenzaremos importando las librerías necesarias:

In [1]:
#TEST_CELL
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 9.1 MB/s eta 0:00:00


In [2]:
import re
import spacy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from unidecode import unidecode
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

Ahora cargamos el conjunto de datos:

In [ ]:
#TEST_CELL
data = (
        pd.read_parquet("https://raw.githubusercontent.com/mindlab-unal/mlds4-datasets/main/u5/poems.parquet")
        .dropna()
        )
display(data.head())

Como podemos ver, el conjunto tiene columnas:

- `author`: Nombre del autor del poema.
- `content`: Texto el poema.
- `title`: Título del poema.

Vamos a preprocesar el conjunto de datos:

In [ ]:
nlp = spacy.blank("es")
def preprocess(text):
    doc = nlp(text) # creamos un documento de spacy
    no_stops = " ".join(
        token.text
        for token in filter(
            lambda token: not token.is_stop and len(token) > 3 and len(token) < 24,
            doc,
            )
        ) # eliminamos stopwords y palabras por longitud
    norm_text = unidecode(no_stops.lower()) # normalizamos el texto
    no_chars = re.sub(r"[^a-z ]", " ", norm_text) # eliminamos caracteres especiales
    no_spaces = re.sub(r"\s+", " ", no_chars) # eliminamos espacios duplicados
    striped_text = no_spaces.strip()
    if not len(striped_text):
        return None
    else:
        return striped_text

Aplicamos la función de preprocesamiento:

In [ ]:
data = (
        data
        .assign(
            corpus=data.content.apply(preprocess)
            )
        .dropna()
        )

Inspeccionemos el tamaño de este conjunto de datos:

In [ ]:
#TEST_CELL
display(data.shape)

## **1. Extracción de Características**
---

En este punto deberá codificar de forma numérica el corpus. Para ello, deberá entrenar un vectorizador TF-IDF con sublinear scaling que permita extraer únicamente los términos que aparecen por lo menos en el 0.5% de los documentos en el corpus.

Para esto, deberá implementar la función `vectorizer` la cual recibirá el corpus preprocesado y deberá retornar un arreglo de `numpy` con la representación y el vectorizador.

**Parámetros**

- `corpus`: `pd.Series` con los textos preprocesados del conjunto de datos.

**Retorna**:

- `features`: arreglo de numpy con la representación de tipo TF-IDF.
- `vect`: `TfidfVectorizer` entrenado con las especificaciones dadas.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pistas</b></font>
</summary>

- Recuerde que _sublinear scaling_ se puede controlar con el parámetro `sublinear_tf` del vectorizador.
- Recuerde convertir la representación a un arreglo de `numpy`.
- Puede usar el parámetro `min_df` para filtrar términos por frecuencia de documento.
</details>

In [ ]:
# FUNCIÓN CALIFICADA vectorizer:
from sklearn.feature_extraction.text import TfidfVectorizer

def vectorizer(corpus):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    vect = ...
    features = ...
    return features, vect
    ### FIN DEL CÓDIGO ###

In [ ]:
#TEST_CELL
features, vect = vectorizer(data.corpus)
display(features.shape)

**Salida esperada**:

En este primer ejemplo debe obtener el tamaño de la representación:

```python
❱ display(features.shape)
(5125, 2232)
```

In [ ]:
#TEST_CELL
features, vect = vectorizer(data.corpus)
display(vect.get_feature_names_out()[:5])

**Salida esperada**:

En este caso deberá obtener las primeras 5 palabras del vocabulario:

```python
❱ display(vect.get_feature_names_out()[:5])
array(['abajo', 'abandonado', 'abandono', 'abeja', 'abejas'], dtype=object)
```

In [ ]:
#TEST_CELL
features, vect = vectorizer(data.corpus)
display(features.sum())

**Salida esperada**:

En este caso deberá obtener la suma de toda la matriz:

```python
❱ display(features.sum())
26109.80862778348
```

## **2. Modelo de Agrupamiento**
---

En este punto deberá entrenar un modelo de K-Means y evaluar el coeficiente de silueta para un número específico de clusters $K$.

Para esto, deberá implementar la función `clustering`, la cual recibirá una matriz de características y deberá retornar el modelo entrenado y el valor del coeficiente de silueta.

**Parámetros**

- `features`: arreglo de `numpy` con las características de los textos.
- `n_clusters`: número de clusters a usar.
- `seed`: semilla de números aleatorios.

**Retorna**:

- `model`: modelo de K-Means entrenado.
- `score`: valor del coeficiente de silueta.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pistas</b></font>
</summary>

- Recuerde que puede controlar el número de clusters con el parámetro `n_clusters`.
- Recuerde que el coeficiente de silueta no necesita ninguna etiqueta, la función recibe las características y las predicciones del modelo.
</details>

In [ ]:
# FUNCIÓN CALIFICADA clustering:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

def clustering(features, n_clusters, seed):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    model = ...
    score = ...
    return model, score
    ### FIN DEL CÓDIGO ###

In [ ]:
#TEST_CELL
features, vect = vectorizer(data.corpus)
model, score = clustering(
        features=features,
        n_clusters=5,
        seed=0
        )
display(score)

**Salida esperada**:

El coeficiente de silueta debería dar un resultado igual a:

```python
❱ display(score)
0.0011509503589058696
```

In [ ]:
#TEST_CELL
features, vect = vectorizer(data.corpus)
model, score = clustering(
        features=features,
        n_clusters=10,
        seed=0
        )
display(score)

**Salida esperada**:

El coeficiente de silueta debería dar un resultado igual a:

```python
❱ display(score)
0.0024804861748867956
```

## **3. Documento Más Relevante**
---

En este punto deberá encontrar el documento más similar a un cluster en específico. El proceso debe seguir los siguientes pasos:

1. Calcular la similitud coseno entre las características de cada documento y el centroide de un cluster dado.
2. Encontrar el id del documento con mayor similitud coseno.
3. Extraer el documento del corpus.

Para esto deberá implementar la función `cluster_document`, la cual toma como entrada el corpus, las características, un modelo entrenado y el id de un cluster. Esta función debe retornar el texto del documento más relevante.

**Parámetros**

- `corpus`: `pd.Series` con el texto preprocesado.
- `features`: arreglo de `numpy` con las características de los textos.
- `model`: modelo de K-Means entrenado.
- `cluster_id`: identificador del cluster a analizar.

**Retorna**:

- `relevant_doc`: documento más relevante para el cluster en cuestión.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pistas</b></font>
</summary>

- Puede acceder a los centroides del modelo K-Means con el atributo `cluster_centers_` del modelo entrenado.
- Puede usar la función `np.argmax` para encontrar el documento más similar.
</details>

In [ ]:
# FUNCIÓN CALIFICADA cluster_document:
from sklearn.metrics.pairwise import cosine_similarity

def cluster_document(corpus, features, model, cluster_id):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    relevant_doc = ...
    return relevant_doc
    ### FIN DEL CÓDIGO ###

In [ ]:
#TEST_CELL
features, vect = vectorizer(data.corpus)
model = clustering(features, 25, 0)[0]
relevant_doc = cluster_document(data.content, features, model, 0)
print(relevant_doc)

**Salida esperada**:

Este primer ejemplo debería retornar el documento más relevante para el cluster 0.

```python
❱ print(relevant_doc)
Cien sonetos de amor

«Vendrás conmigo» ?dije? sin que nadie supiera
dónde y cómo latía mi estado doloroso,
y para mí no había clavel ni barcarola,
nada sino una herida por el amor abierta.
Repetí: ven conmigo, como si me muriera,
y nadie vio en mi boca la luna que sangraba,
nadie vio aquella sangre que subía al silencio.
Oh amor ahora olvidemos la estrella con espinas!
Por eso cuando oí que tu voz repetía
«Vendrás conmigo» ?fue como si desataras
dolor, amor, la furia del vino encarcelado
que desde su bodega sumergida subiera
y otra vez en mi boca sentí un sabor de llama,
de sangre y de claveles, de piedra y quemadura.
```

In [ ]:
#TEST_CELL
features, vect = vectorizer(data.corpus)
model = clustering(features, 25, 0)[0]
relevant_doc = cluster_document(data.content, features, model, 2)
print(relevant_doc)

**Salida esperada**:

Este segundo ejemplo debería retornar el documento más relevante para el cluster 2.

```python
❱ print(relevant_doc)

Eres uno con Dios, porque le amas.
¡Tu pequeñez qué importa y tu miseria,
eres uno con Dios, porque le amas!
Le buscaste en los libros,
le buscaste en los templos,
le buscaste en los astros,
y un día el corazón te dijo, trémulo:
«aquí está», y desde entonces ya sois uno,
ya sois uno los dos, porque le amas.
No podrían separaros
ni el placer de la vida
ni el dolor de la muerte.
En el placer has de mirar su rostro,
en el dolor has de mirar su rostro,
en vida y muerte has de mirar su rostro.
«¡Dios!» dirás en los besos,
dirás «Dios» en los cantos,
dirás «¡Dios!» en los ayes.
Y comprendiendo al fin que es ilusorio
todo pecado (como toda vida),
y que nada de Él puede separarte,
uno con Dios te sentirás por siempre:
uno solo con Dios, porque le amas.
```

## **4. Modelo de Tópicos**
---

En este punto deberá entrenar un modelo de *Latent Semantic Analysis* sobre el corpus de poemas.

Para esto deberá implementar la función `topic_model`, la cual toma como entrada las características del texto y el número de tópicos. Esta deberá retornar el modelo entrenado.

**Nota**: debe utilizar el algoritmo `arpack` en `TruncatedSVD` para que los resultados sean consistentes, es decir, `algorithm="arpack"` como argumento del modelo.

**Parámetros**

- `features`: arreglo de `numpy` con las características de los textos.
- `n_components`: número de tópicos.

**Retorna**:

- `model`: modelo de tópicos entrenado.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pistas</b></font>
</summary>

- Recuerde especificar el algorithmo de optimización, de lo contrario los resultados pueden ser variables.
- Puede especificar el número de tópicos con el parámetro `n_components` del modelo.
</details>

In [ ]:
# FUNCIÓN CALIFICADA topic_model:
from sklearn.decomposition import TruncatedSVD

def topic_model(features, n_components):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    model = ...
    return model
    ### FIN DEL CÓDIGO ###

In [ ]:
#TEST_CELL
features, vect = vectorizer(data.corpus)
model = topic_model(features, 10)
display(model.components_[:5, :5])

**Salida esperada**:

Este ejemplo debería mostrar las primeras 5 filas y las primeras 5 columnas de la matriz tópico-término.

```python
❱ print(model.components_[:5, :5])
array([[ 0.02037573,  0.01055891,  0.01040283,  0.00990592,  0.00902456],
       [-0.01477555, -0.00338276, -0.00075354, -0.01209799, -0.00512086],
       [-0.02064902, -0.01047121, -0.01311938,  0.01244566,  0.00910363],
       [-0.0211558 ,  0.00615672,  0.01806387, -0.00759171, -0.00587832],
       [ 0.01138   ,  0.00010993, -0.01209617, -0.00322183,  0.00915466]])
```

In [ ]:
#TEST_CELL
features, vect = vectorizer(data.corpus)
model = topic_model(features, 2)
display(model.components_[:, :5])

**Salida esperada**:

Este ejemplo debería mostrar las primeras las primeras 5 columnas de la matriz tópico-término.

```python
❱ display(model.components_[:5])
array([[ 0.02037573,  0.01055891,  0.01040283,  0.00990592,  0.00902456],
       [-0.01477555, -0.00338276, -0.00075354, -0.01209799, -0.00512086]])
```

## **5. Documento Más Relevante**
---

En este punto deberá extraer el documento más relevante de un tópico específico. El proceso debe seguir los siguientes pasos:

1. Extraer la matriz documento-tópico y sacar su valor absoluto.
2. Extraer la columna correspondiente al identificador de un tópico dado.
3. Encontrar el identificador del documento con mayor valor dentro de la columna del tópico.
4. Retornar el texto del documento correspondiente.

Para ello deberá implementar la función `topic_document`, la cual toma como entrada las características de los textos, el corpus, el modelo entrenado y un identificador de tópico y debe retornar el documento más relevante para dicho tópico.

**Parámetros**

- `features`: arreglo de `numpy` con las características de los textos.
- `corpus`: `pd.Series` con los documentos.
- `model`: modelo de tópicos entrenado.
- `topic_id`: identificador del tópico.

**Retorna**:

- `relevant_doc`: documento relevante para el tópico en cuestión.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pistas</b></font>
</summary>

- La matriz documento-tópico se puede extraer con el método `transform` del modelo.
- Puede usar la función `np.argmax` para encontrar el documento más relevante en un tópico.
</details>

In [ ]:
# FUNCIÓN CALIFICADA topic_document:
def topic_document(features, corpus, model, topic_id):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    relevant_doc = ...
    return relevant_doc
    ### FIN DEL CÓDIGO ###

In [ ]:
#TEST_CELL
features, vect = vectorizer(data.corpus)
model = topic_model(features, 10)
relevant_doc = topic_document(features, data.content, model, 1)
print(relevant_doc)

**Salida esperada**:

Este ejemplo debería mostrar el poema más relacionado al tópico 1:

```python
❱ print(relevant_doc)
Dios mío, yo te ofrezco mi dolor:
¡Es todo lo que puedo ya ofrecerte!
Tú me diste un amor, un solo amor,
¡un gran amor!
Me lo robó la muerte
...y no me queda más que mi dolor.
Acéptalo, Señor:
¡Es todo lo que puedo ya ofrecerte!...
```

In [ ]:
#TEST_CELL
features, vect = vectorizer(data.corpus)
model = topic_model(features, 10)
relevant_doc = topic_document(features, data.content, model, 2)
print(relevant_doc)

**Salida esperada**:

Este ejemplo debería mostrar el poema más relacionado al tópico 2:

```python
❱ print(relevant_doc)
Cien sonetos de amor

Mi fea, eres una castaña despeinada,
mi bella, eres hermosa como el viento,
mi fea, de tu boca se pueden hacer dos,
mi bella, son tus besos frescos como sandías.
Mi fea, dónde están escondidos tus senos?
Son mínimos como dos copas de trigo.
Me gustaría verte dos lunas en el pecho:
las gigantescas torres de tu soberanía.
Mi fea, el mar no tiene tus uñas en su tienda,
mi bella, flor a flor, estrella por estrella,
ola por ola, amor, he contado tu cuerpo:
mi fea, te amo por tu cintura de oro,
mi bella, te amo por una arruga en tu frente,
amor, te amo por clara y por oscura.
```

## Créditos
---

* **Profesor:** [Felipe Restrepo Calle](https://dis.unal.edu.co/~ferestrepoca/)
* **Asistentes docentes:**
    - [Juan Sebastián Lara Ramírez](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/).
* **Diseño de imágenes:**
    - [Rosa Alejandra Superlano Esquibel](mailto:rsuperlano@unal.edu.co).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*